In [1]:
from pydantic import BaseModel

In [4]:
class Person(BaseModel):
    first_name : str
    last_name: str

In [5]:
p = Person(first_name = "test", last_name = "person")

In [6]:
p

Person(first_name='test', last_name='person')

### Deserializing dict to the model 

In [7]:
data = { "first_name" : "test", "last_name" : "person" }

In [9]:
p = Person.model_validate(data)

In [10]:
p

Person(first_name='test', last_name='person')

### Serializing model to dict 

In [11]:
p.model_dump()

{'first_name': 'test', 'last_name': 'person'}

### Serializing model to json 

In [13]:
p.model_dump_json()

'{"first_name":"test","last_name":"person"}'

### Optional fields and Nullable fields

In [24]:
from typing import Union

class Person(BaseModel):
    first_name : str = "test"
    last_name: str
    ethinicity: Union[str,None] = None

In [25]:
p = Person(last_name="person")

In [26]:
p

Person(first_name='test', last_name='person', ethinicity=None)

### Aliases 

In [27]:
from pydantic import Field

class Person(BaseModel):
    first_name : str = Field(alias="firstName",default="test")
    last_name: str
    ethinicity: Union[str,None] = None

In [28]:
data = { "firstName" : "test", "last_name" : "person" }

In [29]:
p = Person.model_validate(data)

In [30]:
p

Person(first_name='test', last_name='person', ethinicity=None)

In [34]:
data = { "first_name" : "test", "last_name" : "person" }

In [35]:
p = Person.model_validate(data)

In [36]:
p

Person(first_name='test', last_name='person', ethinicity=None)